In [99]:
from linares_plot import *
import os
import datetime
from datetime import datetime as dt
import re


In [114]:
df_gea=pd.read_excel(os.getcwd() + '\\EVENTOS_GEA.xlsx', sheet_name='Hoja2' )
times_s = [df_gea.tiempo.iloc[i].minute*60 + df_gea.tiempo.iloc[i].second for i in range(len(df_gea))]
df_gea['times_s'] = times_s

### cambiar tiempo y tiempo_s por los corregidos, en los que se restan 10 minutos, 31 segundos. De esta manera, 
### los dos dataframes están sincronizados

new_times=[]
times_s_new=[]

for i in range(len(df_gea)):
    #print(i)
    h = df_gea.tiempo.iloc[i]
    h_t = str(h.hour) + ':' + str(h.minute) + ':' + str(h.second) 
    substracting = '00:10:31'
    new_time = dt.strptime(h_t, "%H:%M:%S") - dt.strptime(substracting, "%H:%M:%S")
    new_times.append(new_time)
    ##
    times_s_new.append(df_gea.times_s.loc[i] - (60*10 + 31) ) 

    
df_gea['tiempo'] = new_times 
df_gea['times_s'] = times_s_new 

df_gea = df_gea.sort_values(by=['times_s'])

df_gea= df_gea.reset_index()
df_gea = df_gea[['tiempo', 'evento', 'decripcion', 'times_s']]
df_gea['evento_'] = df_gea['evento'].copy()
df_gea['evento_'] = df_gea['evento_'].replace(['Miniclimax', 'Climax final' ], ['climax', 'climax']) #for simplicity
df_gea.head()

,tiempo,evento,decripcion,times_s,evento_
0,00:00:27,gag,broma sobre su dominio del catalán,27,gag
1,00:00:34,gag,"""Por deferencia""",34,gag
2,00:00:46,inicio,Abre el estuche de la baraja,46,inicio
3,00:01:05,gag,"""Que haya público""",65,gag
4,00:01:19,gag,"""El público de schroedinger""",79,gag


In [140]:
## seleccionar truco
T='Truco 1'
df=pd.read_excel(os.getcwd() + '\\ANOTACIONES2.xlsx', sheet_name= T )

In [139]:
df.head()

,tiempo,1a,1b,2a,2b,3a,3b,4a,4b,5a,...,33a,33b,34a,34b,35a,35b,36a,37a,38a,38b
0,00:06:24,N,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,...,NaN,NaN,a3,NaN,a4,NaN,NaN,NaN,NaN,NaN
1,00:06:26,NaN,NaN,B,NaN,B,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN
2,00:06:28,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,...,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00:06:30,NaN,NaN,S,NaN,P,NaN,NaN,NaN,NaN,...,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00:06:32,S,NaN,R,NaN,NaN,NaN,NaN,NaN,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
list_columns_subjects = list(df.columns)
list_columns_subjects.remove('tiempo')

##list_columns_subjects=list_columns_subjects[70:84] ## para ir comprobando, una a una, que no haya espacios vacíos!

frames=[]

for subject in list_columns_subjects:
    #print(subject)
    dfs = df[['tiempo', subject]]
    dfs = dfs.loc[~pd.isna(dfs[subject])] # Quitar todos los NaN (qudarnos solo con los eventos)
    ####################################### Añadir una columna de segundos
    times_s=[]
    for i in range(0, len(dfs)):
        l = dfs.tiempo.iloc[i]
        times_s.append(  int(datetime.timedelta(hours=l.hour, minutes=l.minute, seconds=l.second).total_seconds()) )
    #
    dfs.columns=['tiempo', 'event']
    dfs['time_s'] = times_s
    ####################################### Añadir columna de subject y de truco
    dfs['trick'] = int(T.split(' ')[-1]) ## last character of the string is the number of the trick
    dfs['subject'] = int(re.findall('\d+', subject)[0]) ##separate leter from number
    frames.append(dfs)
    
    
dfs=pd.concat(frames)

In [153]:
df_subjects=[]

for subject in dfs.subject.unique():
    #######################################################
    df_x = dfs.loc[dfs['subject']==subject]
    df_x = df_x.sort_values(by=['time_s'])
    df_x['duration']=1
    df_x = df_x.reset_index(drop=True) 
    ######################################################
    boole=[] ###bolean to get the final of actions
    for i in range(len(df_x)):
        stri = df_x['event'].iloc[i]
        boole.append( stri[-1]=='*' )
    #
    boole=np.array(boole)
    finals = df_x[boole]
    for fi in range(len(finals)):
        event_final = finals.event.iloc[fi][:-1] ##without the astherisc
        time_final = finals.time_s.iloc[fi]
        #
        previous_event = df_x.loc[(df_x['event']==event_final) & (df_x['time_s']<time_final) ].iloc[-1] ##get the last one
        duration_event = time_final - previous_event.time_s
        ## Change the duration of the dataframe
        df_x.at[previous_event.name, 'duration'] = duration_event
    #
    df_x = df_x[~boole] ### remove the events with astherist
    ######################################################
    boole_apl=[] ###bolean to get the final of actions
    for i in range(len(df_x)):
        stri = df_x['event'].iloc[i]
        if stri[0]=='a':
            boole_apl.append(True)
        elif stri[0]=='A':
            boole_apl.append(True)
        else:
            boole_apl.append(False)
    #
    boole_apl=np.array(boole_apl)    
    aplauses = df_x[boole_apl]
    for ap in range(len(aplauses)):
        event_ = aplauses.iloc[ap] 
        name_event = event_.event
        duration_app = int(re.findall('\d+', name_event)[0])
        evnt = re.split('(\d+)', name_event)[0]
        #
        df_x.at[event_.name, 'duration'] = duration_app
        df_x.at[event_.name, 'event'] = evnt
    #
    ######################################################
    prev_event=[]
    times_after=[]
    for evnt in range(len(df_x)):
        time_ = df_x.iloc[evnt].time_s    
        previous_events_gea = df_gea.loc[df_gea['times_s']<=time_]
        if len(previous_events_gea)==0:
            prev_event.append('None')
            times_after.append(-1)
        else:
            previous_event=previous_events_gea.evento.iloc[-1]
            prev_event.append(previous_event)
            time_after = df_x.time_s.iloc[evnt] - previous_events_gea.times_s.iloc[-1]
            times_after.append(time_after)
    ##
    df_x['RT'] = times_after
    df_x['previous_event'] = prev_event
    ######################################################
    next_event=[]
    times_before=[]
    for evnt in range(len(df_x)): 
        time_ = df_x.iloc[evnt].time_s    
        posts_events_gea = df_gea.loc[df_gea['times_s']>time_]
        if len(previous_events_gea)==0:
            next_event.append('None')
            times_before.append(1)
        else:
            next_evnt=posts_events_gea.evento.iloc[0]
            next_event.append(next_evnt)
            time_before = df_x.time_s.iloc[evnt] - posts_events_gea.times_s.iloc[0]
            times_before.append(time_before)
    ##
    df_x['anticip_time'] = times_before
    df_x['next_event'] = next_event
    ######################################################
    df_subjects.append(df_x)
    

    
    

In [154]:
df_subjects=pd.concat(df_subjects)

In [155]:
df_subjects

,tiempo,event,time_s,trick,subject,duration,RT,previous_event,anticip_time,next_event
0,00:00:06,Se,6,1,1,1,-1,None,-21,gag
1,00:00:10,Si,10,1,1,1,-1,None,-17,gag
2,00:00:12,N,12,1,1,102,-1,None,-15,gag
3,00:00:32,In,32,1,1,1,5,gag,-2,gag
4,00:01:02,In,62,1,1,1,16,inicio,-3,gag
5,00:01:32,In,92,1,1,1,1,gag,-27,gag
6,00:01:48,L,108,1,1,1,17,gag,-11,gag
8,00:01:54,L,114,1,1,1,23,gag,-5,gag
9,00:02:02,N,122,1,1,2,3,gag,-20,gag
11,00:02:08,N,128,1,1,20,9,gag,-14,gag


In [103]:
df_x = dfs.loc[dfs['subject']==3]
df_x = df_x.sort_values(by=['time_s'])
df_x['duration']=1
df_x = df_x.reset_index(drop=True) 
df_x

,tiempo,event,time_s,trick,subject,duration
0,00:00:02,a4,2,1,3,1
1,00:00:06,O,6,1,3,1
2,00:00:14,S,14,1,3,1
3,00:00:22,O,22,1,3,1
4,00:00:26,R,26,1,3,1
5,00:00:28,O*,28,1,3,1
6,00:00:48,P,48,1,3,1
7,00:00:52,L,52,1,3,1
8,00:01:04,S,64,1,3,1
9,00:01:18,P,78,1,3,1


In [104]:
boole=[] ###bolean to get the final of actions
for i in range(len(df_x)):
    stri = df_x['event'].iloc[i]
    boole.append( stri[-1]=='*' )

boole=np.array(boole)    
    
finals = df_x[boole]
finals

,tiempo,event,time_s,trick,subject,duration
5,00:00:28,O*,28,1,3,1


In [105]:
for fi in range(len(finals)):
    event_final = finals.event.iloc[fi][:-1] ##without the astherisc
    time_final = finals.time_s.iloc[fi]
    #
    previous_event = df_x.loc[(df_x['event']==event_final) & (df_x['time_s']<time_final) ].iloc[-1] ##get the last one
    duration_event = time_final - previous_event.time_s
    ## Change the duration of the dataframe
    df_x.at[previous_event.name, 'duration'] = duration_event
    
##
df_x = df_x[~boole] ### remove the events with astherist

In [106]:
boole_apl=[] ###bolean to get the final of actions
for i in range(len(df_x)):
    stri = df_x['event'].iloc[i]
    if stri[0]=='a':
        boole_apl.append(True)
    elif stri[0]=='A':
        boole_apl.append(True)
    else:
        boole_apl.append(False)
    
boole_apl=np.array(boole_apl)    
    
aplauses = df_x[boole_apl]
aplauses

,tiempo,event,time_s,trick,subject,duration
0,00:00:02,a4,2,1,3,1
43,00:05:56,A7,356,1,3,1
45,00:06:18,A+6,378,1,3,1


In [107]:
for ap in range(len(aplauses)):
    event_ = aplauses.iloc[ap] 
    name_event = event_.event
    duration_app = int(re.findall('\d+', name_event)[0])
    evnt = re.split('(\d+)', name_event)[0]
    #
    df_x.at[event_.name, 'duration'] = duration_app
    df_x.at[event_.name, 'event'] = evnt
    

In [108]:
df_x

,tiempo,event,time_s,trick,subject,duration
0,00:00:02,a,2,1,3,4
1,00:00:06,O,6,1,3,1
2,00:00:14,S,14,1,3,1
3,00:00:22,O,22,1,3,6
4,00:00:26,R,26,1,3,1
6,00:00:48,P,48,1,3,1
7,00:00:52,L,52,1,3,1
8,00:01:04,S,64,1,3,1
9,00:01:18,P,78,1,3,1
10,00:01:18,S,78,1,3,1


In [132]:
prev_event=[]
times_after=[]

for evnt in range(len(df_x)): ##len(df_x)
    time_ = df_x.iloc[evnt].time_s    
    previous_events_gea = df_gea.loc[df_gea['times_s']<=time_]
    if len(previous_events_gea)==0:
        prev_event.append('None')
        times_after.append(-1)
    else:
        previous_event=previous_events_gea.evento.iloc[-1]
        prev_event.append(previous_event)
        time_after = df_x.time_s.iloc[evnt] - previous_events_gea.times_s.iloc[-1]
        times_after.append(time_after)

##
df_x['RT'] = times_after
df_x['previous_event'] = prev_event

C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [133]:
df_x.head(10)

,tiempo,event,time_s,trick,subject,duration,RT,previous_event,anticip_time,next_event
0,00:00:02,a,2,1,3,4,-1,None,-25,gag
1,00:00:06,O,6,1,3,1,-1,None,-21,gag
2,00:00:14,S,14,1,3,1,-1,None,-13,gag
3,00:00:22,O,22,1,3,6,-1,None,-5,gag
4,00:00:26,R,26,1,3,1,-1,None,-1,gag
6,00:00:48,P,48,1,3,1,2,inicio,-17,gag
7,00:00:52,L,52,1,3,1,6,inicio,-13,gag
8,00:01:04,S,64,1,3,1,18,inicio,-1,gag
9,00:01:18,P,78,1,3,1,13,gag,-1,gag
10,00:01:18,S,78,1,3,1,13,gag,-1,gag


In [136]:
next_event=[]
times_before=[]

for evnt in range(len(df_x)): ##len(df_x)
    time_ = df_x.iloc[evnt].time_s    
    posts_events_gea = df_gea.loc[df_gea['times_s']>time_]
    if len(previous_events_gea)==0:
        next_event.append('None')
        times_before.append(1)
    else:
        next_evnt=posts_events_gea.evento.iloc[0]
        next_event.append(next_evnt)
        time_before = df_x.time_s.iloc[evnt] - posts_events_gea.times_s.iloc[0]
        times_before.append(time_before)

##
df_x['anticip_time'] = times_before
df_x['next_event'] = next_event

C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [137]:
df_x

,tiempo,event,time_s,trick,subject,duration,RT,previous_event,anticip_time,next_event
0,00:00:02,a,2,1,3,4,-1,None,-25,gag
1,00:00:06,O,6,1,3,1,-1,None,-21,gag
2,00:00:14,S,14,1,3,1,-1,None,-13,gag
3,00:00:22,O,22,1,3,6,-1,None,-5,gag
4,00:00:26,R,26,1,3,1,-1,None,-1,gag
6,00:00:48,P,48,1,3,1,2,inicio,-17,gag
7,00:00:52,L,52,1,3,1,6,inicio,-13,gag
8,00:01:04,S,64,1,3,1,18,inicio,-1,gag
9,00:01:18,P,78,1,3,1,13,gag,-1,gag
10,00:01:18,S,78,1,3,1,13,gag,-1,gag


In [131]:
df_gea


,tiempo,evento,decripcion,times_s,evento_
0,00:00:27,gag,broma sobre su dominio del catalán,27,gag
1,00:00:34,gag,"""Por deferencia""",34,gag
2,00:00:46,inicio,Abre el estuche de la baraja,46,inicio
3,00:01:05,gag,"""Que haya público""",65,gag
4,00:01:19,gag,"""El público de schroedinger""",79,gag
5,00:01:31,gag,El juego es muy bonito,91,gag
6,00:01:59,gag,"No hay humor, la gente se cambia de sitio",119,gag
7,00:02:22,gag,"Broma: ""Ah! No se puede cambiar""",142,gag
8,00:02:30,gag,"""Como veis soy una persona…""",150,gag
9,00:02:48,gag,"""Preocupado por el ritmo…""",168,gag


In [301]:
previous_event.name

136

In [286]:
df_x = df_x.reset_index()
df_x

ValueError: cannot insert level_0, already exists

In [281]:
df_x.iloc[previous_event.name, 'duration']=25

IndexError: single positional indexer is out-of-bounds

In [271]:
df_x.loc[(df_x['event']==event_final) & (df_x['time_s']<time_final), 'duration' ].iloc[-1] 

0

In [276]:
df_x.loc[(df_x['event']==event_final) & (df_x['time_s']<time_final)].iloc[-1].duration

0

In [274]:
df_x

,tiempo,event,time_s,trick,subject,duration
3,00:00:06,Se,6,1,1,0
5,00:00:10,Si,10,1,1,0
6,00:00:12,N,12,1,1,0
16,00:00:32,In,32,1,1,0
31,00:01:02,In,62,1,1,0
46,00:01:32,In,92,1,1,0
54,00:01:48,L,108,1,1,0
57,00:01:54,N*,114,1,1,0
57,00:01:54,L,114,1,1,0
61,00:02:02,N,122,1,1,0


In [255]:
previous_event = df_x.loc[(df_x['event']=='N') & (df_x['time_s']<114) ].iloc[-1]

tiempo     00:00:12
event             N
time_s           12
trick             1
subject           1
Name: 6, dtype: object

'A8'

In [37]:
l = df.tiempo.iloc[158### hacer columna de tiempo en segundos
times_s=[]
for i in range(0, len(df)):
    l = df.tiempo.iloc[158]
    times_s.append(  int(datetime.timedelta(hours=l.hour, minutes=l.minute, seconds=l.second).total_seconds()) )



df['time_s'] = times_s
df.head()]
print(l.hour)
print(l.minute)
print(l.second)

0
5
16


In [38]:
l = df.tiempo.iloc[158]
int(datetime.timedelta(hours=l.hour, minutes=l.minute, seconds=l.second).total_seconds())

316

In [11]:
from datetime import datetime

datetime.strptime(df.tiempo.iloc[0], "%H:%M:%S")

TypeError: strptime() argument 1 must be str, not datetime.time